In [193]:
import networkx as nx
from networkx.algorithms import approximation
import numpy as np 
import copy
from scipy import linalg
from scipy.optimize import minimize
import scipy
from multiprocessing import Pool

In [194]:
def make_elist(N):
    elist = []
    for i in range(N-1):
        for j in range(i+1, N):
            elist.append((i, j))
    elist.pop(-1)
    return elist

In [195]:
def ansatz(params, sigma_x, n, half, len_InS, atomwise):
        gammas = params[:half]
        betas = params[half:]
        state = np.zeros(len_InS)
        state[0] = 1
        if atomwise: 
            for p in range(len(gammas)):
                for i in range(len(sigma_x)):
                    state = scipy.sparse.linalg.expm_multiply(-1j*(gammas[p]*sigma_x[i]+betas[p]*n[i]), state)
        else:
            for p in range(len(gammas)):
                for i in range(len(sigma_x)):
                    state = scipy.sparse.linalg.expm_multiply(-1j*gammas[p]*sigma_x[i], state)
                for i in range(len(sigma_x)):
                    state = scipy.sparse.linalg.expm_multiply(-1j*betas[p]*n[i], state)
        return -np.real(np.dot(np.conjugate(state), sum(n)*state))

In [201]:
def QAOA(X): 
    p, N = X
    G = nx.Graph(make_elist(N))
    GC = nx.algorithms.operators.complement(G)
    InS = [np.zeros(N)]
    for clique in nx.algorithms.clique.enumerate_all_cliques(GC): 
        new_InS = np.zeros(N)
        for node in clique: 
            new_InS[node] = 1
        InS.append(new_InS)
    InS = np.array(InS)
    len_InS = len(InS)
    MIS_N = np.max(np.sum(InS, axis=1))
    sigma_x = np.zeros((N, len_InS, len_InS))
    n = np.zeros((N, len_InS, len_InS))
    for i in range(N):
        for j in range(len_InS):
            n[i][j][j] = InS[j][i]
    n = [scipy.sparse.csr_matrix(i) for i in n]
    for i in range(N):
        for j in range(len_InS):
            tolookfor = copy.deepcopy(InS[j])
            tolookfor[i] = abs(tolookfor[i]-1)
            k = np.where((InS == tolookfor).all(axis=1))[0]
            if k.size>0:
                sigma_x[i][j][k[0]] = 1
    sigma_x = [scipy.sparse.csr_matrix(i) for i in sigma_x]
    half = int(p)
    bounds = np.array([(-np.pi, np.pi) for _ in range(2*p)])
    best_atom = (0, None)
    best = (0, None)
    for _ in range(10):
        p0 = np.random.uniform(-np.pi, np.pi, 2*p)
        atomwise = 1
        result_atom = minimize(ansatz, p0, args=(sigma_x, n, half, len_InS, atomwise), bounds=bounds, method="L-BFGS-B")
        atomwise = 0
        result = minimize(ansatz, p0, args=(sigma_x, n, half, len_InS, atomwise), bounds=bounds, method="L-BFGS-B")
        if -result_atom['fun']/MIS_N > best_atom[0]:
            best_atom = (-result_atom['fun']/MIS_N, result_atom['x'])
        if -result['fun']/MIS_N > best[0]:
            best = (-result['fun']/MIS_N, result['x'])
    print(N, p, best_atom, best)

In [202]:
toloop = []
for N in range(3, 5):
    for p in range(1, N-1):
        toloop.append((p, N))
pool = Pool(processes=63)
pool.map(QAOA, toloop)

1 3 (0.5624999999999996, array([-1.61818743, -1.63816893])) (0.5624999999999996, array([ 2.09439513, -1.00490027]))
1 4 (0.5185185185185179, array([ 0.95536913, -0.0288949 ])) (0.5185185185185184, array([ 0.9553166 , -0.01828731]))
2 4 (0.8432258905328728, array([ 1.95902111, -0.99459248,  2.6962079 ,  0.57421919])) (0.9084236169837637, array([ 2.51482992, -2.13268251,  3.14159265,  0.53135167]))


[None, None, None]

In [ ]:
for N in range(10, 12, 2):
    p=0
    accuracy = [0.0]
    while abs(accuracy[0]-1.0)>1e-2:
        p+=1
        for _ in range(100):
            accuracy = parallel(p, N)
            if(accuracy[0]!=1.0):
                print(N, p, parallel2(p, N))

In [ ]:
for p in range(6, 10): 
    X = parallel(p)
    accuracy = []
    optimizationtime = []
    print(p, X[0], X[1], X[2])

In [ ]:
def parallel2(num_p, N): 
    G = nx.random_regular_graph(3, N)
    GC = nx.algorithms.operators.complement(G)
    InS = [np.zeros(N)]
    for clique in nx.algorithms.clique.enumerate_all_cliques(GC): 
        toappend = np.zeros(N)
        for node in clique: 
            toappend[node] = 1
        InS.append(toappend)
    InS = np.array(InS)
    MIS_N = np.max(np.sum(InS, axis=1))
    sigma_x = np.zeros((N, len(InS), len(InS)))
    n = np.zeros((N, len(InS), len(InS)))
    for i in range(N):
        for j in range(len(InS)):
            n[i][j][j] = InS[j][i]
    for i in range(N):
        for j in range(len(InS)):
            tolookfor = copy.deepcopy(InS[j])
            tolookfor[i] = abs(tolookfor[i]-1)
            k = np.where((InS == tolookfor).all(axis=1))[0]
            if k.size>0:
                sigma_x[i][j][k[0]] = 1
    def ansatz(params):
        half = int(len(params)/2)
        gammas = params[:half]
        betas = params[half:]
        state = np.zeros((len(InS),1))
        state[0] = 1
        for p in range(len(gammas)):
            for i in range(N):
                state = np.matmul(linalg.expm(-1j*gammas[p]*sigma_x[i]), state)
            for i in range(N):
                state = np.matmul(linalg.expm(1j*betas[p]*n[i]), state)
        return state
    def print_state(state):
        print(state)
    def ansatz_print(params):
        half = int(len(params)/2)
        gammas = params[:half]
        betas = params[half:]
        state = np.zeros((len(InS),1))
        state[0] = 1
        #print_state(state)
        for p in range(len(gammas)):
            for i in range(N):
                state = np.matmul(linalg.expm(-1j*gammas[p]*sigma_x[i]), state)
                #print_state(state)
            for i in range(N):
                state = np.matmul(linalg.expm(1j*betas[p]*n[i]), state)
                #print_state(state)
        return state
    def obj_function(state):
        return np.matmul(state.conjugate().transpose(), np.matmul(sum(n), state))
    def func(params):
        return -np.real(obj_function(ansatz(params))[0])[0]
    final_params = None
    final_val = 10000
    for just in range(100):
        result = minimize(func, [np.random.uniform(0, np.pi/2.0) for _ in range(2*num_p)],  bounds = [(0, np.pi/2.0) for _ in range(2*num_p)], method="L-BFGS-B", options={'maxiter': 1e4})
        if result['success']==0:
            print("NO")
        if result['fun'] < final_val:
            final_params = result['x']
            final_val = result['fun']
        solution = ansatz_print(result['x'])
        prob = [np.abs(comple)**2.0 for comple in solution]
        accuracy= 0
        for ind in range(len(InS)):
            if sum(InS[ind])==MIS_N:
                accuracy+= prob[ind]
        print(int(just/100.0*100), end='\r')
    solution = ansatz_print(final_params)
    prob = [np.abs(comple)**2.0 for comple in solution]
    accuracy= 0
    for ind in range(len(InS)):
        if sum(InS[ind])==MIS_N:
            accuracy+= prob[ind]
    return accuracy, final_params

    